In [3]:
import mysql.connector
import pandas as pd

In [7]:
csv_files = [
( 'Attendance.csv','Attendance'),
('Hiring_Funnel.csv','Hiring_Funnel'),
('Emp_master.csv','Emp_master'),
('dept.csv','dept')]

In [8]:
conn = mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

In [13]:
cursor= conn.cursor()
import os

In [14]:
folder_path = 'C:/Users/gracy/Downloads/endd'

In [15]:
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing Attendance.csv
NaN values before replacement:
LogID             0
EmployeeID        0
Date              0
Status            0
HoursWorked       0
Shift          2715
dtype: int64

Processing Hiring_Funnel.csv
NaN values before replacement:
CandidateID            0
DepartmentApplied      0
SourceChannel          0
ApplicationDate        0
Stage                  0
DropoutReason        754
dtype: int64

Processing Emp_master.csv
NaN values before replacement:
EmployeeID    0
Name          0
Department    0
JobRole       0
HireDate      0
Status        0
dtype: int64

Processing dept.csv
NaN values before replacement:
DepartmentID      0
DepartmentName    0
dtype: int64



In [2]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

db =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = db.cursor()


In [3]:
query="SELECT * FROM attendance;"
df=pd.read_sql(query,db)

In [4]:
df.head()

,LogID,EmployeeID,Date,Status,HoursWorked,Shift
0,1,1,01-01-2023,Present,8,Morning
1,2,2,01-01-2023,Present,6,Evening
2,3,3,01-01-2023,Present,7,Morning
3,4,4,01-01-2023,Present,6,Morning
4,5,5,01-01-2023,Present,9,Evening


In [5]:
query="SELECT * FROM dept;"
df=pd.read_sql(query,db)

df.head()


,DepartmentID,DepartmentName
0,1,HR
1,2,IT
2,3,Finance
3,4,Sales
4,5,Marketing


In [6]:
query="SELECT * FROM hiring_funnel;"
df=pd.read_sql(query,db)

df.head()


,CandidateID,DepartmentApplied,SourceChannel,ApplicationDate,Stage,DropoutReason
0,1,Marketing,LinkedIn,19-02-2023,Applied,None
1,2,Sales,Career Fair,24-01-2023,Applied,None
2,2,Sales,Career Fair,24-01-2023,Screened,None
3,3,Sales,Career Fair,26-02-2023,Applied,None
4,3,Sales,Career Fair,26-02-2023,Screened,None


In [7]:
query="SELECT * FROM emp_master;"
df=pd.read_sql(query,db)

df.head(20)


,EmployeeID,Name,Department,JobRole,HireDate,Status
0,1,Employee_1,IT,System Admin,09-07-2022,Active
1,2,Employee_2,Marketing,SEO Specialist,30-03-2022,Active
2,3,Employee_3,Marketing,SEO Specialist,08-03-2021,Terminated
3,4,Employee_4,HR,HR Manager,28-09-2021,Terminated
4,5,Employee_5,Finance,Financial Analyst,23-07-2021,Active
5,6,Employee_6,Marketing,SEO Specialist,30-06-2022,Terminated
6,7,Employee_7,HR,HR Manager,07-01-2021,Active
7,8,Employee_8,Sales,Sales Executive,10-02-2022,Active
8,9,Employee_9,IT,Developer,28-12-2021,Terminated
9,10,Employee_10,Finance,Financial Analyst,15-09-2021,Terminated


# Profiling

In [8]:
# Total EMPLOYEES
df = pd.read_sql("SELECT COUNT(*) AS TOTAL_EMP FROM EMP_MASTER;",db)
print("TOTAL_EMP:",df.iloc[0,0])
# Distribution of employees by department (text join not needed here)
query = "SELECT department, COUNT(*) AS emp_count FROM emp_master GROUP BY department;"
df = pd.read_sql(query, db)
print(df)
# Check for missing values in key columns
query = """
SELECT 
    SUM(CASE WHEN employeeid IS NULL THEN 1 ELSE 0 END) AS missing_employee_id,
    SUM(CASE WHEN department IS NULL THEN 1 ELSE 0 END) AS missing_department
FROM emp_master;
"""
df = pd.read_sql(query, db)
print(df)

TOTAL_EMP: 100
  department  emp_count
0         IT         19
1  Marketing         15
2         HR         19
3    Finance         31
4      Sales         16
   missing_employee_id  missing_department
0                  0.0                 0.0


In [9]:
# Total attendance records
query = "SELECT COUNT(*) AS total_attendance FROM attendance;"
df = pd.read_sql(query, db)
print(df)

# Date range
query = "SELECT MIN(date) AS start_date, MAX(date) AS end_date FROM attendance;"
df = pd.read_sql(query, db)
print(df)

# Status distribution (Present, Absent, WFH)
query = "SELECT status, COUNT(*) AS count FROM attendance GROUP BY status;"
df = pd.read_sql(query, db)
print(df)

# Missing values
query = """
SELECT
    SUM(CASE WHEN employeeid IS NULL THEN 1 ELSE 0 END) AS missing_employee_id,
    SUM(CASE WHEN date IS NULL THEN 1 ELSE 0 END) AS missing_date,
    SUM(CASE WHEN status IS NULL THEN 1 ELSE 0 END) AS missing_status
FROM attendance;
"""
df = pd.read_sql(query, db)
print(df)

   total_attendance
0              8897
   start_date    end_date
0  01-01-2023  31-03-2023
       status  count
0     Present   6182
1      Absent   1417
2         WFH    847
3  Sick Leave    451
   missing_employee_id  missing_date  missing_status
0                  0.0           0.0             0.0


In [10]:
# Total candidates
query = "SELECT COUNT(*) AS total_candidates FROM hiring_funnel;"
df = pd.read_sql(query, db)
print(df)

# Candidates per stage
query = "SELECT stage, COUNT(*) AS count FROM hiring_funnel GROUP BY stage;"
df = pd.read_sql(query, db)
print(df)

# Missing values
query = """
SELECT
    SUM(CASE WHEN candidateid IS NULL THEN 1 ELSE 0 END) AS missing_candidate_id,
    SUM(CASE WHEN stage IS NULL THEN 1 ELSE 0 END) AS missing_stage
FROM hiring_funnel;
"""
df = pd.read_sql(query, db)
print(df)

   total_candidates
0               866
       stage  count
0    Applied    300
1   Screened    234
2  Interview    164
3      Offer    116
4      Hired     52
   missing_candidate_id  missing_stage
0                   0.0            0.0


In [11]:
# Total departments
query = "SELECT COUNT(*) AS total_departments FROM dept;"
df = pd.read_sql(query, db)
print(df)

# List departments
query = "SELECT * FROM dept;"
df = pd.read_sql(query, db)
print(df)

   total_departments
0                  5
   DepartmentID DepartmentName
0             1             HR
1             2             IT
2             3        Finance
3             4          Sales
4             5      Marketing


WORKFORCE COMPOSITION

In [101]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)

query = """
SELECT 
    em.Department,
    em.Status,
    COUNT(*) AS employee_count,
    ROUND(AVG(DATEDIFF(CURDATE(), STR_TO_DATE(em.HireDate, '%Y-%m-%d')) / 365), 1) AS avg_tenure_years
FROM Emp_Master em
WHERE em.HireDate IS NOT NULL
GROUP BY em.Department, em.Status
ORDER BY em.Department, em.Status;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Dept","Status","Emp Count","Avg Tenure Years"])
df

,Dept,Status,Emp Count,Avg Tenure Years
0,Finance,Active,15,6.4
1,Finance,Terminated,16,9.7
2,HR,Active,12,9.1
3,HR,Terminated,7,10.8
4,IT,Active,10,11.6
5,IT,Terminated,9,9.9
6,Marketing,Active,8,13.8
7,Marketing,Terminated,7,7.8
8,Sales,Active,10,9.1
9,Sales,Terminated,6,9.2


In [102]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)
query = """
SELECT 
    em.Department,
    COUNT(*) AS total_employees,
    SUM(CASE WHEN em.Status = 'Active' THEN 1 ELSE 0 END) AS active_employees
FROM Emp_Master em
GROUP BY em.Department
ORDER BY total_employees DESC;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Dept","Total Emp","Active Emp"])
df

,Dept,Total Emp,Active Emp
0,Finance,31,15
1,IT,19,10
2,HR,19,12
3,Sales,16,10
4,Marketing,15,8


In [103]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)
query = """
SELECT 
    CASE 
        WHEN ROUND(DATEDIFF(CURDATE(), STR_TO_DATE(HireDate, '%Y-%m-%d')) / 365, 1) < 1 THEN '0-1 years'
        WHEN ROUND(DATEDIFF(CURDATE(), STR_TO_DATE(HireDate, '%Y-%m-%d')) / 365, 1) BETWEEN 1 AND 3 THEN '1-3 years'
        WHEN ROUND(DATEDIFF(CURDATE(), STR_TO_DATE(HireDate, '%Y-%m-%d')) / 365, 1) BETWEEN 3 AND 5 THEN '3-5 years'
        ELSE '5+ years'
    END AS tenure_range,
    COUNT(*) AS employee_count
FROM Emp_Master
GROUP BY tenure_range
ORDER BY tenure_range;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Tenure_Range","employee_count"])
df

,Tenure_Range,employee_count
0,0-1 years,25
1,1-3 years,9
2,3-5 years,5
3,5+ years,61


ATTENDANCE & LEAVE ANALYSIS

In [104]:
query="""
SELECT 
    sub.department,
    AVG(sub.sick_leave_count) AS avg_sick_leaves
FROM (
    SELECT 
        a.employeeid,
        e.department,
        SUM(CASE WHEN a.status = 'Sick Leave' THEN 1 ELSE 0 END) AS sick_leave_count
    FROM attendance a
    JOIN emp_master e 
        ON a.employeeid = e.employeeid
    GROUP BY a.employeeid, e.department
) sub
GROUP BY sub.department
ORDER BY avg_sick_leaves DESC;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Department","Avg_sick_leave"])
df

,Department,Avg_sick_leave
0,Marketing,5.0667
1,IT,4.7895
2,HR,4.4211
3,Sales,4.3750
4,Finance,4.1935


In [105]:
query = """SELECT 
    a.employeeid,
    COUNT(CASE WHEN a.status = 'Present' THEN 1 END) AS present_days,
    COUNT(CASE WHEN a.status = 'Absent' THEN 1 END) AS absent_days,
    COUNT(CASE WHEN a.status = 'Sick Leave' THEN 1 END) AS sick_leave_days,
    COUNT(CASE WHEN a.status = 'WFH' THEN 1 END) AS wfh_days
FROM attendance a
GROUP BY a.employeeid
ORDER BY  sick_leave_days desc  ;"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["emp","present","Absent","Sick leave","WFH"])
df

,emp,present,Absent,Sick leave,WFH
0,96,57,14,8,10
1,72,62,10,8,9
2,4,60,14,8,7
3,57,65,12,8,5
4,36,56,16,8,9
...,...,...,...,...,...
95,54,62,17,1,9
96,88,65,14,1,9
97,90,65,10,1,13
98,95,60,20,1,9


In [106]:
query = """ SELECT SHIFT_STATUS , TOTAL_RECORDS FROM (SELECT 
    CASE 
        WHEN status IN ('Absent', 'Sick Leave') THEN 'No Shift (Leave/Absent)'
        WHEN status = 'WFH' THEN 'Work From Home'
        ELSE shift
    END AS shift_status,
    
    SUM(CASE WHEN status = 'Present' THEN 1 ELSE 0 END) AS present_days,
    SUM(CASE WHEN status = 'WFH' THEN 1 ELSE 0 END) AS wfh_days,
    SUM(CASE WHEN status = 'Sick Leave' THEN 1 ELSE 0 END) AS sick_leave_days,
    SUM(CASE WHEN status = 'Absent' THEN 1 ELSE 0 END) AS absent_days,
    
    COUNT(*) AS total_records
FROM attendance
GROUP BY 
    CASE 
        WHEN status IN ('Absent', 'Sick Leave') THEN 'No Shift (Leave/Absent)'
        WHEN status = 'WFH' THEN 'Work From Home'
        ELSE shift
    END
ORDER BY shift_status) AS FINAL;"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["SHIFT","NUMBER_OF_DAYS"])
df

,SHIFT,NUMBER_OF_DAYS
0,Evening,3112
1,Morning,3070
2,No Shift (Leave/Absent),1868
3,Work From Home,847


In [107]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)

query = """SELECT 
    DATE_FORMAT(STR_TO_DATE(a.date, '%Y-%m-%d'), '%Y-%m') AS month,
    COUNT(DISTINCT CASE WHEN a.status = 'Present' THEN a.employeeid END) AS present_emp,
    COUNT(DISTINCT CASE WHEN a.status = 'Absent' THEN a.employeeid END) AS absent_emp,
    COUNT(DISTINCT CASE WHEN a.status = 'Sick Leave' THEN a.employeeid END) AS sick_leave_emp,
    COUNT(DISTINCT CASE WHEN a.status = 'WFH' THEN a.employeeid END) AS wfh_emp
FROM attendance a
GROUP BY month
ORDER BY month;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Month","Present_emp","Absent_emp","Sick_leave_emp","WFH_emp"])
df



,Month,Present_emp,Absent_emp,Sick_leave_emp,WFH_emp
0,2001-01,71,18,7,4
1,2001-02,67,14,7,10
2,2001-03,67,19,7,6
3,2002-01,76,13,3,7
4,2002-02,72,15,2,9
...,...,...,...,...,...
85,2029-03,68,16,6,9
86,2030-01,65,22,3,8
87,2030-03,67,15,4,13
88,2031-01,62,20,9,7


HIRING FUNNEL

In [108]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)
query = """
SELECT
    stage,
    COUNT(DISTINCT candidateid) AS total_dropouts
FROM hiring_funnel
WHERE dropoutreason IS NOT NULL
GROUP BY stage
ORDER BY total_dropouts DESC;
SELECT
    stage,
    COUNT(DISTINCT candidateid) AS total_dropouts
FROM hiring_funnel
WHERE dropoutreason IS NOT NULL
GROUP BY stage
ORDER BY total_dropouts DESC;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Stage","Total Dropout"])
df

,Stage,Total Dropout
0,Applied,39
1,Interview,31
2,Screened,31
3,Offer,11


In [109]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)
query = """
SELECT
    stage,
    dropoutreason,
    COUNT(DISTINCT candidateid) AS count
FROM hiring_funnel
WHERE dropoutreason IS NOT NULL
GROUP BY stage, dropoutreason
ORDER BY stage, count DESC;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Stage","Dropout Reason","Emp Count"])
df

,Stage,Dropout Reason,Emp Count
0,Applied,Rejected,15
1,Applied,Better Offer,13
2,Applied,Not Interested,11
3,Interview,Rejected,14
4,Interview,Better Offer,12
5,Interview,Not Interested,5
6,Offer,Better Offer,6
7,Offer,Rejected,3
8,Offer,Not Interested,2
9,Screened,Rejected,12


In [110]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)
query = """SELECT 
    SourceChannel, 
    count(distinct candidateid) as total_applicants,
    count(distinct case when stage = "hired" then candidateid end) as hired,
    round(count(distinct case when stage = "hired" then candidateid end)*100/
    count(distinct candidateid),2) as hire_percentage,
    count(distinct case when dropoutreason is not null then candidateid end) as dropouts,
    round(count(distinct case when dropoutreason is not null then candidateid end)*100/
     count(distinct candidateid),2 ) as dropout_percentage
     from hiring_funnel
GROUP BY SourceChannel
order by  count(distinct candidateid) desc"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["Source Channel","Total Applicants","Employee Hired","Hire Percentage","Dropouts","Drop Percentage"])
df

,Source Channel,Total Applicants,Employee Hired,Hire Percentage,Dropouts,Drop Percentage
0,Referral,79,16,20.25,26,32.91
1,Career Fair,77,11,14.29,25,32.47
2,Job Portal,74,11,14.86,26,35.14
3,LinkedIn,70,14,20.00,24,34.29


In [111]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "HR"
)

cur = conn.cursor(buffered=True)
query = """SELECT
    stage,
    COUNT(DISTINCT candidateid) AS total_candidates,
    ROUND(
        CASE 
            WHEN stage = 'Applied' THEN NULL
            WHEN stage = 'Screened' THEN COUNT(DISTINCT CASE WHEN stage = 'Screened' THEN candidateid END) * 100.0 /
                                        (SELECT COUNT(DISTINCT candidateid) FROM hiring_funnel)
            WHEN stage = 'Interview' THEN COUNT(DISTINCT CASE WHEN stage = 'Interview' THEN candidateid END) * 100.0 /
                                           (SELECT COUNT(DISTINCT candidateid) FROM hiring_funnel WHERE stage = 'Screened')
            WHEN stage = 'Offer' THEN COUNT(DISTINCT CASE WHEN stage = 'Offer' THEN candidateid END) * 100.0 /
                                       (SELECT COUNT(DISTINCT candidateid) FROM hiring_funnel WHERE stage = 'Interview')
            WHEN stage = 'Hired' THEN COUNT(DISTINCT CASE WHEN stage = 'Hired' THEN candidateid END) * 100.0 /
                                     (SELECT COUNT(DISTINCT candidateid) FROM hiring_funnel WHERE stage = 'Offer')
        END, 2
    ) AS conversion_percentage
FROM hiring_funnel
GROUP BY stage
ORDER BY 
    CASE stage
        WHEN 'Applied' THEN 1
        WHEN 'Screened' THEN 2
        WHEN 'Interview' THEN 3
        WHEN 'Offer' THEN 4
        WHEN 'Hired' THEN 5
    END;"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["Stage","Total Candidates","Conversion Percentage"])
df
 

,Stage,Total Candidates,Conversion Percentage
0,Applied,300,None
1,Screened,234,78.00
2,Interview,164,70.09
3,Offer,116,70.73
4,Hired,52,44.83
